In [ ]:
import cv2
from os.path import basename
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageChops

In [ ]:
def map_mask(imarray):
    gray = cv2.cvtColor(imarray, cv2.COLOR_BGR2GRAY)  # greyscale image
    # Detect Background Color
    pix_hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    background_pix_value = np.argmax(pix_hist, axis=None)

    # Flood fill borders
    height, width = gray.shape[:2]
    corners = [[0,0],[0,height-1],[width-1, 0],[width-1, height-1]]
    for c in corners:
        cv2.floodFill(gray, None, (c[0],c[1]), 255)

    # AdaptiveThreshold to remove noise
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 4)

    # Edge Detection
    thresh_blur = cv2.GaussianBlur(thresh, (11, 11), 0)
    canny = cv2.Canny(thresh_blur, 0, 200)
    canny_dilate = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))

    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny_dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    
    # Keeping only the largest detected contour.
    contour = sorted(contours, key=cv2.contourArea, reverse=True)[0]

    # Mask out everything that is not the map
    mask = np.zeros_like(imarray)
    cv2.fillPoly(mask, pts=[contour], color=(255,255,255))
    masked_img = cv2.bitwise_and(imarray, mask)

    return masked_img

In [ ]:
filePath = '/home/shared/DARPA/training/'
trainSource = 'AK_McCarthy.tif'
sourceImg = cv2.imread(filePath+trainSource)

In [ ]:
maskedImg = map_mask(sourceImg)

fig, axs = plt.subplots(1, 2, figsize=(15, 8))
axs[0].imshow(sourceImg)
axs[1].imshow(maskedImg)

plt.show()